# FastAPI NLP API Demo

This notebook demonstrates how to interact with the NLP API endpoints.

In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import websocket
import threading
from IPython.display import display, HTML, JSON

In [2]:
# API configuration
BASE_URL = "http://localhost:8000/api/v1"
# Default credentials (update these)
USERNAME = "admin"
PASSWORD = "password"

## 1. Authentication

First, let's authenticate with the API to get a token.

In [3]:
def get_token(username, password):
    """Get authentication token from the API."""
    response = requests.post(
        f"{BASE_URL}/token",
        data={"username": username, "password": password},
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    
    if response.status_code == 200:
        token_data = response.json()
        return token_data["access_token"]
    else:
        print(f"Authentication failed with status code {response.status_code}")
        print(response.text)
        return None

In [ ]:
# Get the authentication token
token = get_token(USERNAME, PASSWORD)

if token:
    print(f"Authentication successful! Token obtained.")
    # Set up the default headers with our token
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
else:
    print("Failed to get token. Please check your credentials.")

## 2. Health Check

Let's check if the API is healthy and all models are available.

In [ ]:
def check_health():
    """Check the health status of the API."""
    response = requests.get(f"{BASE_URL}/health")
    return response.json()

health_status = check_health()
display(JSON(health_status))

# Visual representation of model availability
if "models" in health_status:
    models_df = pd.DataFrame([{"model": k, "available": v} for k, v in health_status["models"].items()])
    plt.figure(figsize=(10, 5))
    sns.barplot(x="model", y="available", data=models_df)
    plt.title("Model Availability")
    plt.ylabel("Available")
    plt.xlabel("Model")
    plt.show()

## 3. Get User Information

Let's check the current user's information.

In [ ]:
def get_user_info():
    """Get current user information."""
    response = requests.get(
        f"{BASE_URL}/users/me",
        headers=headers
    )
    return response.json()

user_info = get_user_info()
display(JSON(user_info))

## 4. Sentiment Analysis

Let's analyze the sentiment of some example texts.

In [7]:
def analyze_sentiment(text, language="en"):
    """Analyze the sentiment of the given text."""
    payload = {
        "text": text,
        "language": language
    }
    
    response = requests.post(
        f"{BASE_URL}/sentiment",
        headers=headers,
        json=payload
    )
    
    return response.json()

In [ ]:
# Test with some example texts
example_texts = [
    "I love this product, it's amazing!",
    "This is the worst experience ever.",
    "The package arrived on time.",
    "I'm not sure what to think about this."
]

sentiment_results = []
for text in example_texts:
    result = analyze_sentiment(text)
    sentiment_results.append(result)
    display(JSON(result))
    
# Create a visualization of the sentiment scores
sentiment_df = pd.DataFrame([
    {"text": r["text"], "sentiment": r["sentiment"], "score": r["score"]} 
    for r in sentiment_results
])

plt.figure(figsize=(10, 6))
bars = sns.barplot(x="text", y="score", hue="sentiment", data=sentiment_df)
plt.title("Sentiment Analysis Results")
plt.ylabel("Confidence Score")
plt.xlabel("")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## 5. Named Entity Recognition

Let's extract entities from some sample texts.

In [9]:
def extract_entities(text):
    """Extract named entities from the given text."""
    payload = {"text": text}
    
    response = requests.post(
        f"{BASE_URL}/entities",
        headers=headers,
        json=payload
    )
    
    return response.json()

In [ ]:
ner_text = "Apple is looking to buy U.K. startup for $1 billion in January 2023. Microsoft CEO Satya Nadella announced the news in New York."
entities = extract_entities(ner_text)
display(JSON(entities))

# Create a visualization of entity types
if entities and "entities" in entities:
    entities_df = pd.DataFrame(entities["entities"])
    
    # Count entities by type
    entity_counts = entities_df["type"].value_counts().reset_index()
    entity_counts.columns = ["entity_type", "count"]
    
    plt.figure(figsize=(10, 5))
    sns.barplot(x="entity_type", y="count", data=entity_counts)
    plt.title("Entity Types in Text")
    plt.ylabel("Count")
    plt.xlabel("Entity Type")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Display the annotated text
    def highlight_entities(text, entities):
        """Highlight entities in text with HTML colors."""
        colors = {
            "PERSON": "#FF9999",
            "ORG": "#99FF99",
            "LOC": "#9999FF",
            "DATE": "#FFFF99",
            "TIME": "#99FFFF",
            "MONEY": "#FF99FF",
            "PERCENT": "#FFCC99",
        }
        
        # Sort entities by start position (reversed to avoid index issues)
        sorted_entities = sorted(entities, key=lambda x: x["start"], reverse=True)
        
        result = text
        for entity in sorted_entities:
            start = entity["start"]
            end = entity["end"]
            entity_type = entity["type"]
            entity_text = text[start:end]
            color = colors.get(entity_type, "#CCCCCC")
            
            highlight = f'<span style="background-color: {color}; padding: 2px; border-radius: 3px;" title="{entity_type}">{entity_text}</span>'
            result = result[:start] + highlight + result[end:]
            
        return result
    
    highlighted_text = highlight_entities(entities["text"], entities["entities"])
    display(HTML(f"<p style='font-size: 16px; line-height: 1.5;'>{highlighted_text}</p>"))

## 6. Entity Filtering

Let's try filtering entities by type and confidence score.

In [11]:
def filter_entities(text, entity_types=None, min_score=0.5):
    """Filter entities by type and minimum confidence score."""
    params = {"text": text, "min_score": min_score}
    
    if entity_types:
        params["entity_types"] = entity_types
    
    response = requests.get(
        f"{BASE_URL}/entities/filter",
        headers=headers,
        params=params
    )
    
    return response.json()

In [ ]:
# Filter only for organizations and persons with high confidence
filtered_entities = filter_entities(
    ner_text, 
    entity_types=["ORG", "PERSON"], 
    min_score=0.7
)

display(JSON(filtered_entities))

# Compare filtered vs total entities count
if "filtered_from" in filtered_entities and "filtered_to" in filtered_entities:
    filter_stats = pd.DataFrame([
        {"category": "Total Entities", "count": filtered_entities["filtered_from"]},
        {"category": "Filtered Entities", "count": filtered_entities["filtered_to"]}
    ])
    
    plt.figure(figsize=(8, 5))
    sns.barplot(x="category", y="count", data=filter_stats)
    plt.title("Entity Filtering Results")
    plt.ylabel("Count")
    plt.xlabel("")
    plt.tight_layout()
    plt.show()

## 7. Text Classification

Let's classify some sample texts into categories.

In [13]:
def classify_text(text, labels, multi_label=False):
    """Classify text into given categories."""
    payload = {
        "text": text,
        "labels": labels,
        "multi_label": multi_label
    }
    
    response = requests.post(
        f"{BASE_URL}/classify",
        headers=headers,
        json=payload
    )
    
    return response.json()

In [ ]:
# Get classification presets
def get_classification_preset(preset_name):
    """Get predefined label sets for classification."""
    response = requests.get(
        f"{BASE_URL}/classify/presets/{preset_name}",
        headers=headers
    )
    
    return response.json()

# Get the topics preset
topics_preset = get_classification_preset("topics")
display(JSON(topics_preset))

In [ ]:
# Example texts for classification
classification_examples = [
    "Climate change poses serious environmental challenges",
    "The latest smartphone features a 108MP camera and advanced AI",
    "The stock market plunged 500 points due to economic concerns",
    "The team won the championship with a last-minute goal"
]

# Use the topics from the preset
topic_labels = topics_preset["labels"]

# Classify each example text
classification_results = []
for text in classification_examples:
    result = classify_text(text, topic_labels)
    classification_results.append(result)
    display(JSON(result))
    
# Visualize top categories for each text
classification_data = []
for result in classification_results:
    # Sort labels by score (highest first) and get top label
    top_label = sorted(result["labels"], key=lambda x: x.get("score", 0), reverse=True)[0]
    classification_data.append({
        "text": result["text"][:30] + "...",  # Truncate long texts
        "top_category": top_label.get("label"),
        "score": top_label.get("score")
    })

class_df = pd.DataFrame(classification_data)
plt.figure(figsize=(12, 6))
bars = sns.barplot(x="text", y="score", hue="top_category", data=class_df)
plt.title("Top Categories for Example Texts")
plt.ylabel("Confidence Score")
plt.xlabel("")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.legend(title="Category")
plt.show()

## 8. Custom Classification with Query Parameters

Let's try the custom classification endpoint that uses query parameters.

In [ ]:
def custom_classify(text, labels):
    """Classify text using query parameters."""
    params = {
        "text": text,
        "labels": labels
    }
    
    response = requests.post(
        f"{BASE_URL}/classify/custom",
        headers=headers,
        params=params
    )
    
    return response.json()

# Try with intent classification
intent_result = custom_classify(
    "How do I reset my password?", 
    ["question", "complaint", "feedback", "request"]
)
display(JSON(intent_result))

## 9. WebSocket Connection

Let's demonstrate the WebSocket API for real-time NLP processing.

In [17]:
class NLPWebSocketClient:
    def __init__(self, url="ws://localhost:8000/api/v1/ws"):
        self.url = url
        self.ws = None
        self.messages = []
        self.connected = False
    
    def on_message(self, ws, message):
        """Callback when a message is received."""
        data = json.loads(message)
        self.messages.append(data)
        print(f"Received: {json.dumps(data, indent=2)}")
    
    def on_error(self, ws, error):
        """Callback for errors."""
        print(f"Error: {error}")
    
    def on_close(self, ws, close_status_code, close_msg):
        """Callback when connection is closed."""
        print("Connection closed")
        self.connected = False
    
    def on_open(self, ws):
        """Callback when connection is established."""
        print("Connection opened")
        self.connected = True
    
    def connect(self):
        """Connect to the WebSocket."""
        self.ws = websocket.WebSocketApp(
            self.url,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close
        )
        
        # Start WebSocket in a background thread
        self.thread = threading.Thread(target=self.ws.run_forever)
        self.thread.daemon = True
        self.thread.start()
        
        # Wait for connection
        time.sleep(1)
        return self.connected
    
    def send(self, data):
        """Send data to the WebSocket."""
        if not self.ws or not self.connected:
            print("Not connected")
            return False
        
        self.ws.send(json.dumps(data))
        return True
    
    def disconnect(self):
        """Disconnect from the WebSocket."""
        if self.ws:
            self.ws.close()
        
        # Wait for thread to finish
        if hasattr(self, 'thread') and self.thread.is_alive():
            self.thread.join(timeout=1)
            
        self.connected = False

In [ ]:
# Create and connect the WebSocket client
ws_client = NLPWebSocketClient()
connected = ws_client.connect()

if connected:
    # Try sentiment analysis
    ws_client.send({"action": "sentiment", "text": "I'm really enjoying this API!"})
    time.sleep(1)  # Wait for response
    
    # Try entity recognition
    ws_client.send({"action": "entities", "text": "Google's headquarters are in Mountain View, California."})
    time.sleep(1)  # Wait for response
    
    # Try classification
    ws_client.send({
        "action": "classify", 
        "text": "This computer keeps crashing whenever I open the browser", 
        "labels": ["technical issue", "feedback", "question"]
    })
    time.sleep(1)  # Wait for response
    
    # Wait a bit to receive all responses then disconnect
    time.sleep(2)
    ws_client.disconnect()

## 10. Admin Actions

Let's try using the admin-only endpoint to reload models.

In [ ]:
def reload_models():
    """Reload NLP models (admin only)."""
    response = requests.post(
        f"{BASE_URL}/admin/reload-models",
        headers=headers
    )
    
    return response.json()

reload_result = reload_models()
display(JSON(reload_result))

## Conclusion

This notebook demonstrates how to interact with all the major endpoints of the NLP API. You can use this as a starting point to build applications that leverage these natural language processing capabilities.

Here's a summary of what we've explored:

1. Authentication and getting tokens
2. API health checks
3. Sentiment analysis
4. Named entity recognition
5. Text classification
6. WebSocket connections for real-time processing
7. Admin actions for system management

These building blocks can be combined to create powerful NLP-based applications.